# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import libraries
import pandas as pd 
import re
import os
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse_table', engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [5]:
# Initialize stopwords and lemmatizer once
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Tokenization function
def tokenize(text):
    # Case normalization & remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stopwords
    words = [w for w in words if w not in stop_words]
    
    # Lemmatize words (first pass, default as noun)
    lemmed = [lemmatizer.lemmatize(w) for w in words]
    
    # Lemmatize as verbs (second pass, treating words as verbs)
    words = [lemmatizer.lemmatize(w, pos='v') for w in lemmed]

    return words

In [6]:
# Test with first 5 messages in X
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['look', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Create pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to word count vectors
    ('tfidf', TfidfTransformer()),  # Convert word count vectors to tf-idf
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))  # Multi-output classifier using DecisionTree
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Predict on the test data
Y_pred__grid = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
# Evaluate classification report for each category
for column in Y.columns:
    print(f"Classification report for {column}:")
    
    # Get the true labels and predicted labels for each category
    print(classification_report(Y_test[column], Y_pred[:, Y.columns.get_loc(column)]), "\n")

Classification report for related:
             precision    recall  f1-score   support

          0       0.51      0.51      0.51      1266
          1       0.84      0.83      0.84      3938
          2       0.26      0.53      0.34        40

avg / total       0.76      0.75      0.76      5244
 

Classification report for request:
             precision    recall  f1-score   support

          0       0.91      0.92      0.91      4349
          1       0.57      0.55      0.56       895

avg / total       0.85      0.85      0.85      5244
 

Classification report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244
 

Classification report for aid_related:
             precision    recall  f1-score   support

          0       0.75      0.76      0.76      3113
          1       0.65      0.63      0.64     

In [11]:
# Ensure consistent shape between Y_test and Y_pred_grid
print(Y_test.shape)

(5244, 36)


In [13]:
print(Y_pred_grid.shape)

(5244, 36)


In [21]:
for i, column in enumerate(Y.columns):
    print(f"Classification report for {column}:")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]), "\n")

Classification report for related:
             precision    recall  f1-score   support

          0       0.51      0.51      0.51      1266
          1       0.84      0.83      0.84      3938
          2       0.26      0.53      0.34        40

avg / total       0.76      0.75      0.76      5244
 

Classification report for request:
             precision    recall  f1-score   support

          0       0.91      0.92      0.91      4349
          1       0.57      0.55      0.56       895

avg / total       0.85      0.85      0.85      5244
 

Classification report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244
 

Classification report for aid_related:
             precision    recall  f1-score   support

          0       0.75      0.76      0.76      3113
          1       0.65      0.63      0.64     

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define parameter grid for tuning DecisionTreeClassifier hyperparameters
param_grid = {
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__min_samples_split': [2, 5],
    'clf__estimator__min_samples_leaf': [1, 2]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=2, n_jobs=-1, verbose=2)

# Fit the grid search model on a sample of training data (for faster results)
X_train_sample = X_train[:1000]  # Use a smaller sample
Y_train_sample = Y_train[:1000]
grid_search.fit(X_train_sample, Y_train_sample)

# Print the best parameters found by the grid search
print("Best parameters found: ", grid_search.best_params_)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5 
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=1, clf__

[Parallel(n_jobs=-1)]: Done   3 out of  16 | elapsed:   18.2s remaining:  1.3min


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5, total=  14.1s
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5, total=  14.2s
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total=  13.6s
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total=  15.2s
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, total=  14.2s
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total=  13.8s
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5, total=  15.6s
[CV]  clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5, total=  14.5s
[CV]  clf__estim

[Parallel(n_jobs=-1)]: Done  12 out of  16 | elapsed:   19.4s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   19.6s finished


Best parameters found:  {'clf__estimator__max_depth': 10, 'clf__estimator__min_samples_leaf': 2, 'clf__estimator__min_samples_split': 2}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

# Use the best model to make predictions on the test set
Y_pred_grid = grid_search.predict(X_test)

# Evaluate the tuned model using classification report
print("Classification report for tuned model:")

# Loop through each column to print the classification report for each category
for i, column in enumerate(Y.columns):
    print(f"Classification report for {column}:")
    print(classification_report(Y_test.iloc[:, i], Y_pred_grid[:, i]), "\n")

# Now let's print accuracy, precision, and recall for each category
print("Accuracy, Precision, and Recall for each category:")

for column in Y.columns:
    accuracy = accuracy_score(Y_test[column], Y_pred_grid[:, Y.columns.get_loc(column)])
    
    # Use 'macro' average for multi-class classification
    precision = precision_score(Y_test[column], Y_pred_grid[:, Y.columns.get_loc(column)], average='macro')
    recall = recall_score(Y_test[column], Y_pred_grid[:, Y.columns.get_loc(column)], average='macro')

    print(f"Category: {column}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision (macro): {precision:.4f}")
    print(f"  Recall (macro): {recall:.4f}")
    print("\n")

Classification report for tuned model:
Classification report for related:
             precision    recall  f1-score   support

          0       0.44      0.12      0.18      1266
          1       0.76      0.95      0.85      3938
          2       0.00      0.00      0.00        40

avg / total       0.68      0.74      0.68      5244
 

Classification report for request:
             precision    recall  f1-score   support

          0       0.88      0.96      0.92      4349
          1       0.67      0.36      0.47       895

avg / total       0.84      0.86      0.84      5244
 

Classification report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 

Classification report for aid_related:
             precision    recall  f1-score   support

          0       0.67      0.90      0.77      3113
        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

# Define the pipeline without additional features
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Random Forest classifier
])

# Define the parameter grid for Random Forest
param_grid = {
    'clf__estimator__n_estimators': [100],  # Keep it smaller
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__min_samples_split': [2],
    'clf__estimator__min_samples_leaf': [1]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=2, n_jobs=-1, verbose=2)

# Fit the grid search model on training data
grid_search.fit(X_train, Y_train)

# Now grid_search will have the best parameters and can be used for further evaluation

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=20, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 1.4min
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 1.4min
[CV]  clf__estimator__max_depth=20, clf__estimator__min_sam

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [100], 'clf__estimator__max_depth': [10, 20], 'clf__estimator__min_samples_split': [2], 'clf__estimator__min_samples_leaf': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 9. Export your model as a pickle file

In [26]:
import pickle

# Save the trained grid_search model to a pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

print("Model has been saved as 'classifier.pkl'")

Model has been saved as 'classifier.pkl'


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
import pandas as pd
import pickle
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Tokenization function
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    words = word_tokenize(text)
    words = [w for w in words if w not in stop_words]
    lemmed = [lemmatizer.lemmatize(w) for w in words]
    words = [lemmatizer.lemmatize(w, pos='v') for w in lemmed]
    return words

# Load data from the database
def load_data(database_filepath):
    engine = create_engine(f'sqlite:///{database_filepath}')
    df = pd.read_sql_table('DisasterResponse_table', engine)
    
    # Define feature and target variables
    X = df['message']
    Y = df.loc[:, 'related':'direct_report']
    category_names = Y.columns.tolist()
    
    return X, Y, category_names

# Build the ML pipeline
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))  # Multi-output classifier using DecisionTree
    ])
    return pipeline

# Evaluate the model
def evaluate_model(model, X_test, Y_test, category_names):
    Y_pred = model.predict(X_test)
    for i, column in enumerate(category_names):
        print(f"Classification report for {column}:")
        print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]), "\n")

# Save the model as a pickle file
def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as f:
        pickle.dump(model, f)

# Main function
def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        
        # Split the data into train and test sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()